# PitViper Notebook Report

In [1]:
from functions_pitviper_nb import * 
import time
alt.renderers.enable('html')

RendererRegistry.enable('html')

In [2]:
# Parameters
mageck_mle_outputs = "../../results/development/MAGeCK_MLE/"
mageck_rra_outputs = "../../results/development/MAGeCK_RRA/"
bagel_outputs = "../../results/development/BAGEL/"
crisphiermix_outputs = "../../results/development/CRISPhieRmix/"

In [3]:
results_path = [mageck_mle_outputs, mageck_rra_outputs, bagel_outputs, crisphiermix_outputs]

results = create_results_pitviper(results_path)

## Gene essentiality analysis

### MAGeCK MLE - Feature accross conditions

In [4]:
feature_accros_conditions(results['MAGeCK_MLE'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J11', 'M07e-J18', 'M07e-J4'), value='M07…

### MAGeCK RRA - Feature accross conditions

In [5]:
feature_accros_conditions(results['MAGeCK_RRA'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J11', 'M07e-J18', 'M07e-J4'), value='M07…

### CRISPhieRmix - Feature accross conditions

In [6]:
feature_accros_conditions(results['CRISPhieRmix'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J11', 'M07e-J18', 'M07e-J4'), value='M07…

### BAGEL - Feature accross conditions

In [7]:
feature_accros_conditions(results['BAGEL'])

interactive(children=(Dropdown(description='baseline', options=('M07e-J11', 'M07e-J18', 'M07e-J4'), value='M07…

## Annotations

### EnrichR

In [8]:
TOOLS = [tool for tool in results.keys()]

BASES = open("../../workflow/notebooks/enrichr_list.txt", "r").readlines()


        
def enrichr_plots(pitviper_res):
    conditions = [{'baseline':condition.split('_vs_')[1] , 'treatment':condition.split('_vs_')[0]} for condition in pitviper_res.comparisons_dict.keys()]
    @interact(description=widgets.Text(value='My gene list', placeholder='Description', description='Description:'), base=BASES, baseline=set([condition['baseline'] for condition in conditions]))
    def enrichr_plots(description, base, baseline):
        treatments = [condition['treatment'] for condition in conditions if condition['baseline'] == baseline]
        @interact(size=[5, 10, 20, 50, 100, 200], tool=TOOLS, treatment=treatments, fdr_cutoff=widgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.05))
        def enrichr_plots(treatment, tool, fdr_cutoff, size):
            comparisons = [condition['treatment']+'_vs_'+baseline for condition in conditions if condition['baseline'] == baseline]
            print('Description:', description)
            print('Baseline:', baseline)
            print('Treatment:', treatment)
            print('FDR cut-off:', fdr_cutoff)
            print('Tool:', tool)
            print('Gene set library:', base)
            
            if tool == 'MAGeCK_MLE':
                info = pitviper_res.comparisons_dict[treatment+'_vs_'+baseline]['table']
                info = info.loc[info[treatment+'|fdr'] < fdr_cutoff]
                genes = info['Gene']

            print("Size (gene set):", len(genes))
            
            def on_button_clicked(b):
                enrichr_res = getEnrichrResults(genes, description, base)
                table = createEnrichrTable(enrichr_res)
                chart = enrichmentBarPlot(table, size, description)
                with output:
                    display(chart)

            
            button = widgets.Button(description="Show EnrichR results")
            output = widgets.Output()

            display(button, output)

            button.on_click(on_button_clicked)

                        
enrichr_plots(results['MAGeCK_MLE'])

interactive(children=(Text(value='My gene list', description='Description:', placeholder='Description'), Dropd…